In [1]:
import pickle
import os
import numpy as np
import numpy as cnp
import matplotlib.pyplot as plt

## Common Metrics

In [2]:
def compute_pdf_metric(pdf_ref, pdf_miw):
    # pdfs [Nx,Ny,Nz]
    return cnp.mean(cnp.abs(pdf_ref-pdf_miw))

def compute_traj_div_metric(trajs_ref, trajs_miw):
    # trajs [numTrajs, numDofUniv] only positions
    return cnp.mean( cnp.linalg.norm(trajs_ref-trajs_miw, axis=-1) )
    # distance from a trajectory in one to the trajectory with the same material label in the other
    
def compute_traj_momentum_metric(mom_field_ref, trajs_miw, dim, cxlowers, cxuppers, cdxs, xs, ys=None, zs=None):
    if dim==3:
        # TRAJ MOMENTA ##################################################
        # if no trajectory surpasses below the node 0 or J-1 at any time,
        # traj will always be among (0,J-1) and traj//dxs will be among [0,J-1]
        trajs_idxs = (((trajs_miw[:,:3]-cxlowers)//cdxs).T).astype(cnp.uint) # [3, numTrajs] the closest index from below along each axis
        # relative distance to the closest index from below point along each dimension
        # the closer, the bigger its weight should be for the trajectory propagation
        ratx_down = ((trajs_miw[:,0]-xs[ trajs_idxs[0] ])/(xs[ trajs_idxs[0]+1 ]-xs[ trajs_idxs[0] ]))[:,cnp.newaxis]
        raty_down = ((trajs_miw[:,1]-ys[ trajs_idxs[1] ])/(ys[ trajs_idxs[1]+1 ]-ys[ trajs_idxs[1] ]))[:,cnp.newaxis]
        ratz_down = ((trajs_miw[:,2]-zs[ trajs_idxs[2] ])/(zs[ trajs_idxs[2]+1 ]-zs[ trajs_idxs[2] ]))[:,cnp.newaxis]
        # Get the interpolated momenta
        correct_momenta = ratx_down*raty_down*ratz_down* mom_field_ref[ trajs_idxs[0]+1, trajs_idxs[1]+1, trajs_idxs[2]+1 ] +\
            (1-ratx_down)*raty_down*ratz_down* mom_field_ref[ trajs_idxs[0], trajs_idxs[1]+1, trajs_idxs[2]+1 ] +\
            ratx_down*(1-raty_down)*ratz_down* mom_field_ref[ trajs_idxs[0]+1, trajs_idxs[1], trajs_idxs[2]+1 ] +\
            ratx_down*raty_down*(1-ratz_down)* mom_field_ref[ trajs_idxs[0]+1, trajs_idxs[1]+1, trajs_idxs[2] ] +\
            (1-ratx_down)*(1-raty_down)*ratz_down* mom_field_ref[ trajs_idxs[0], trajs_idxs[1], trajs_idxs[2]+1 ] +\
            ratx_down*(1-raty_down)*(1-ratz_down)* mom_field_ref[ trajs_idxs[0]+1, trajs_idxs[1], trajs_idxs[2] ] +\
            (1-ratx_down)*raty_down*(1-ratz_down)* mom_field_ref[ trajs_idxs[0], trajs_idxs[1]+1, trajs_idxs[2] ] +\
            (1-ratx_down)*(1-raty_down)*(1-ratz_down)* mom_field_ref[ trajs_idxs[0], trajs_idxs[1], trajs_idxs[2] ]
    elif dim==2:
        # MOMENTUM ON TRAJS TRAJS ##################################################
        # if no trajectory surpasses below the node 0 or J-1 at any time,
        # traj will always be among (0,J-1) and traj//dxs will be among [0,J-1]
        trajs_idxs = (((trajs_miw[:,:2]-cxlowers)//cdxs).T).astype(cnp.uint) # [2, numTrajs] the closest index from below along each axis
        # relative distance to the closest index from below point along each dimension
        # the closer, the bigger its weight should be for the trajectory propagation
        ratx_down = ((trajs_miw[:,0]-xs[ trajs_idxs[0] ])/(xs[ trajs_idxs[0]+1 ]-xs[ trajs_idxs[0] ]))[:,cnp.newaxis]
        raty_down = ((trajs_miw[:,1]-ys[ trajs_idxs[1] ])/(ys[ trajs_idxs[1]+1 ]-ys[ trajs_idxs[1] ]))[:,cnp.newaxis]
        # Interpolate momentum
        correct_momenta = ratx_down*raty_down* mom_field_ref[ trajs_idxs[0]+1, trajs_idxs[1]+1 ] +\
            (1-ratx_down)*raty_down* mom_field_ref[ trajs_idxs[0], trajs_idxs[1]+1] +\
            ratx_down*(1-raty_down)* mom_field_ref[ trajs_idxs[0]+1, trajs_idxs[1] ] +\
            (1-ratx_down)*(1-raty_down)* mom_field_ref[ trajs_idxs[0], trajs_idxs[1]]
    
    else: # dim==1
        trajs_idxs = (((trajs_miw-cxlowers)//cdxs).T).astype(cnp.uint) # [1, numTrajs] the closest index from below along each axis
        # relative distance to the closest index from below point along each dimension
        # the closer, the bigger its weight should be for the trajectory propagation
        ratx_down = ((trajs_miw[:,0]-xs[ trajs_idxs[0] ])/(xs[ trajs_idxs[0]+1 ]-xs[ trajs_idxs[0] ]))[:,cnp.newaxis]
        # Interpolate momentum
        correct_momenta = ratx_down* mom_field_ref[ trajs_idxs[0]+1 ] +\
            (1-ratx_down)* mom_field_ref[ trajs_idxs[0] ]  #[numTrajs,1]

    return cnp.linalg.norm(correct_momenta-trajs_miw[:,dim:], axis=-1).mean()



def compute_mom_expectation_abs_dif(mom_field_ref, pdf_ref, moms_ref, moms_miw):
    # mom_field [Nx,Ny,Nz, 3], pdf [Nx,Ny,Nz], moms_miw and moms_ref [numTrajs, numDofUniv]
    av_mom_ref = []
    total_p = cnp.sum(pdf_ref)
    for j,mom_ref_j in enumerate(cnp.moveaxis(mom_field_ref, -1,0)):
        av_mom_ref.append(cnp.sum(mom_ref_j*pdf_ref)/total_p) 
    av_mom_ref = cnp.array(av_mom_ref) # [numDofUniv]
    av_mom_ref_trajs = cnp.mean(moms_ref, axis=0) # [numDofUniv]
    av_mom_miw = cnp.mean(moms_miw, axis=0) # [numDofUniv]
    return cnp.abs(av_mom_ref-av_mom_miw).mean(), cnp.abs(av_mom_ref_trajs-av_mom_miw).mean()

def compute_posit_expectation_abs_dif(position_field, pdf_ref, poss_ref, poss_miw):
    # position_field [Nx,Ny,Nz, 3], pdf_red [Nx,Ny,Nz], poss_miw and poss_ref [numTrajs, numDofUniv]
    av_pos_ref = []
    total_p = cnp.sum(pdf_ref)
    for pos_ref_j in cnp.moveaxis(position_field,-1,0):
        av_pos_ref.append(cnp.sum(pos_ref_j*pdf_ref)/total_p) 
    av_pos_ref = cnp.array(av_pos_ref) # [numDofUniv]
    av_pos_ref_trajs = cnp.mean(poss_ref, axis=0) # [numDofUniv]
    av_pos_miw = cnp.mean(poss_miw, axis=0) # [numDofUniv]
    return cnp.abs(av_pos_ref-av_pos_miw).mean(), cnp.abs(av_pos_ref_trajs-av_pos_miw).mean()


# Compute metrics and save employed parameters

In [3]:
done_experiment_queue_path = "./QUEUE_done.txt"
experiment_names = []
experiment_grid_files = []
settings_files = []
with open(done_experiment_queue_path, 'r') as f:
    tasks = f.readlines()
for task in tasks:
    args = task.replace(" ", "").split('(')[-1].split(')')[0].split(',')
    experiment_names.append(args[0])
    experiment_grid_files.append(args[-2])
    settings_files.append(args[1])
print(experiment_names)
print(experiment_grid_files)
print(settings_files)

['2D_double_slit', '2D_harmonic_dance', '2D_harmonic_gs', '2D_harmonic_rot_gs', '2D_torus_collision', '2D_torus_dance', '1D_double_barrier_p3', '1D_double_barrier_p1', '1D_double_barrier_p2', '1D_free_particle_p1', '1D_free_particle_p0', '1D_harmonic_gs', '1D_self_interference', '1D_harmonic_dance', '3D_harmonic_dance', '3D_harmonic_gs', '3D_free_particle_p0', '3D_hydrogen_2pz', '3D_hydrogen_gs', '1D_single_barrier_p1', '1D_single_barrier_p2', '1D_single_barrier_p3', '2D_free_particle_p0', '2D_single_slit_p1', '2D_single_slit_p2', '2D_single_slit_p3']
['./SIMULATION_GRID_SETTINGS/SETTINGS_Grid_2d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_2d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_2d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_2d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_2d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_2d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_1d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_1d.txt', './SIMULATION_GRID_SETTINGS/SETTI

In [4]:
done = []

In [7]:
for experiment_name, grid_file, path_to_settings in zip(experiment_names, experiment_grid_files, settings_files):
    print(f"Processing {experiment_name}")
    if experiment_name in done:
        print("Processed already!")
        continue
    outputs_directory = f"./OUTPUTS/{experiment_name}/"
    if not os.path.isfile(f"{outputs_directory}/FINISHED.txt"):
        print(f"Still not finished!")
        continue
    # if results are all done:
    results = {}
    results['Experiment_IDs'] = []
    with open(f"{grid_file}", "r") as f:
        results['numK'] = int(f.readline().split("numK ")[1])
        results['numA'] = int(f.readline().split("numA ")[1])
        results['Krange'] = [float(x) for x in f.readline().split("Krange ")[1].split('[')[1].split(']')[0].split(',')]
        results['Arange'] = [float(x) for x in f.readline().split("Arange ")[1].split('[')[1].split(']')[0].split(',')]
        dim = int(f.readline().split("dim ")[1])
    if len(results['Krange'])>2:
        Ks_to_try = results['Krange']
        As_to_try = results['Arange']
    else:
        Ks_to_try = np.linspace(results['Krange[0]'], results['Krange[1]'], results['numK'])
        As_to_try = np.linspace(results['Arange[0]'], results['Arange[1]'], results['numA'])
    for K in Ks_to_try:
        for A in As_to_try:
            ID = f"K_{K:.4}_A_{A:.4}"
            results['Experiment_IDs'].append( ID )
            results[ID] = {}
            results[ID]['K'] = K
            results[ID]['A'] = A
            results[ID]['pdf_difs_w_ref'] = []
            results[ID]['traj_difs_w_ref'] = []
            results[ID]['mom_difs_w_ref'] = []
            results[ID]['mom_expect_difs_c_w_pdf'] = []
            results[ID]['mom_expect_difs_c_w_trajs'] = []
            results[ID]['posit_expect_difs_w_pdf'] = []
            results[ID]['posit_expect_difs_w_trajs'] = []
    results['average_metrics_wrt_ref'] = {}
    results['average_metrics_wrt_ref']['pdf_dif_w_ref'] = []
    results['average_metrics_wrt_ref']['traj_dif_w_ref'] = []
    results['average_metrics_wrt_ref']['mom_dif_w_ref'] = []
    results['average_metrics_wrt_ref']['mom_expect_dif_c_w_pdf'] = []
    results['average_metrics_wrt_ref']['mom_expect_dif_c_w_trajs'] = []
    results['average_metrics_wrt_ref']['posit_expect_dif_w_pdf'] = []
    results['average_metrics_wrt_ref']['posit_expect_dif_w_trajs'] = []
        
            
    with open(f"{path_to_settings}", "r") as f:
        results['numIts_SE'] = int(f.readline().split("numIts_SE ")[1])
        results['dt_SE'] = float(f.readline().split("dt_SE ")[1])
        results['outputEvery_SE'] = int(f.readline().split("outputEvery_SE ")[1])
        results['Ns_pdf'] = [int(x) for x in f.readline().split("Ns_SE_and_Newt_pdf ")[1].split('[')[1].split(']')[0].split(',')]
        results['xlowers'] = [float(x) for x in f.readline().split("xlowers ")[1].split('[')[1].split(']')[0].split(',')]
        results['xuppers'] = [float(x) for x in f.readline().split("xuppers ")[1].split('[')[1].split(']')[0].split(',')]
        results['numDofUniv'] = int(f.readline().split("numDofUniv ")[1])
        results['numDofPartic'] = int(f.readline().split("numDofPartic ")[1])
        results['numTrajs'] = int(f.readline().split("numTrajs ")[1])
        results['hbar'] = float(f.readline().split("hbar ")[1])
        results['ms'] = [float(x) for x in f.readline().split("ms ")[1].split('[')[1].split(']')[0].split(',')]
        results['complex_dtype'] = f.readline().split("complex_dtype ")[1]
        results['real_dtype'] =  f.readline().split("real_dtype ")[1]
        results['K_coulomb'] = float(f.readline().split("K_coulomb ")[1])
        results['qs'] = [float(x) for x in f.readline().split("qs ")[1].split('[')[1].split(']')[0].split(',')]
        results['dt_MIW'] = float(f.readline().split("dt_Newt ")[1])
        results['Ns_potential'] = [int(x) for x in f.readline().split("Ns_Newt_pot ")[1].split('[')[1].split(']')[0].split(',')]
        results['use_Coulomb_potential'] = bool(int(f.readline().split("use_Coulomb_potential_Newt ")[1]))
        results['use_scenario_potential'] = bool(int(f.readline().split("use_scenario_potential_Newt ")[1]))
        results['sigma'] = float(f.readline().split("sigma_pdf_estimation ")[1])
    nodes = [cnp.linspace(results['xlowers'][j], results['xuppers'][j], results['Ns_pdf'][j]) for j in range(results['numDofUniv'])] # (xs, ys)
    if results['numDofUniv']==3:
        grid=cnp.moveaxis(cnp.array(np.meshgrid(*nodes)).swapaxes(1,2),0,-1) #[Nx,Ny,Nz,3]
    elif results['numDofUniv']==2:
        grid=cnp.array(cnp.meshgrid(*nodes)).T #[Nx,Ny, 2]
    elif results['numDofUniv']==1:
        grid=nodes[0][:,cnp.newaxis]
    cxlowers = cnp.array(results['xlowers'])[cnp.newaxis, :]
    cxuppers = cnp.array(results['xuppers'])[cnp.newaxis, :]
    results['dxs'] = [(results['xuppers'][j]-results['xlowers'][j])/(results['Ns_pdf'][j]-1) for j in range(results['numDofUniv'])] # (dx, dy)
    cdxs = cnp.array(results['dxs'])[cnp.newaxis, :]
    
    # Match the output number
    outputEvery = int(results['dt_SE']//results['dt_MIW'])
    # now match exactly the outputed frames
    results['outputEvery_MIW'] = results['outputEvery_SE']*outputEvery
    results['numIts_MIW'] = int(results['numIts_SE']*(results['dt_SE']//results['dt_MIW']))

    its_SE = np.array([j for j in range(results['numIts_SE'])])
    output_its_SE = its_SE[::results['outputEvery_SE']]
    its_MIW = np.array([j for j in range(results['numIts_MIW'])])
    output_its_MIW = its_MIW[::results['outputEvery_MIW']]
    for it_SE, it_MIW in zip(output_its_SE, output_its_MIW):
        # take the reference results
        se_simul_path = outputs_directory+f"/SE_{results['numDofUniv']}D/Reference_SE_Simulation/"
        pdf_ref = np.load(se_simul_path+f"/pdf/pdf_it_{it_SE}.npy") #[Nx,Ny,Nz]
        trajs_ref = np.load(se_simul_path+f"/trajs/trajs_it_{it_SE}.npy") #[numTrajs, 2*dimUniv]
        mom_field_ref = np.load(se_simul_path+f"/moms/momentum_field_it_{it_SE}.npy") #[Nx,Ny, Nz, 3]
        # remove overflows
        over = 10
        mom_field_ref = cnp.nan_to_num( mom_field_ref, nan=0, posinf=over, neginf=-over )
        for ID in results['Experiment_IDs']:
            miw_simul_path = outputs_directory + f"/MIW/{ID}/"
            pdf_miw = np.load(miw_simul_path+f"/pdf/pdf_it_{it_MIW}.npy") #[Nx,Ny,Nz]
            trajs_miw = np.load(miw_simul_path+f"/trajs/trajs_it_{it_MIW}.npy") #[numTrajs, 2*dimUniv]
            results[ID]['pdf_difs_w_ref'].append(compute_pdf_metric(pdf_ref, pdf_miw))
            results[ID]['traj_difs_w_ref'].append(compute_traj_div_metric(trajs_ref[:,:results['numDofUniv']], trajs_miw[:,:results['numDofUniv']]))
            results[ID]['mom_difs_w_ref'].append(compute_traj_momentum_metric(mom_field_ref, trajs_miw, results['numDofUniv'], cxlowers, cxuppers, cdxs, *nodes))
            r = compute_mom_expectation_abs_dif(mom_field_ref, pdf_ref, trajs_ref[:,results['numDofUniv']:], trajs_miw[:,results['numDofUniv']:])
            results[ID]['mom_expect_difs_c_w_pdf'].append(r[0])
            results[ID]['mom_expect_difs_c_w_trajs'].append(r[1])
            r = compute_posit_expectation_abs_dif(grid, pdf_ref, trajs_ref[:,:results['numDofUniv']], trajs_miw[:,:results['numDofUniv']])
            results[ID]['posit_expect_difs_w_pdf'].append(r[0])
            results[ID]['posit_expect_difs_w_trajs'].append(r[1])
    
    for ID in results['Experiment_IDs']:
        results['average_metrics_wrt_ref']['pdf_dif_w_ref'].append( np.mean(results[ID]['pdf_difs_w_ref']))
        results['average_metrics_wrt_ref']['traj_dif_w_ref'].append( np.mean(results[ID]['traj_difs_w_ref']))
        results['average_metrics_wrt_ref']['mom_dif_w_ref'].append( np.mean(results[ID]['mom_difs_w_ref']))
        results['average_metrics_wrt_ref']['mom_expect_dif_c_w_pdf'].append( np.mean(results[ID]['mom_expect_difs_c_w_pdf']))
        results['average_metrics_wrt_ref']['mom_expect_dif_c_w_trajs'].append( np.mean(results[ID]['mom_expect_difs_c_w_trajs']))
        results['average_metrics_wrt_ref']['posit_expect_dif_w_pdf'].append( np.mean(results[ID]['posit_expect_difs_w_pdf']))
        results['average_metrics_wrt_ref']['posit_expect_dif_w_trajs'].append( np.mean(results[ID]['posit_expect_difs_w_trajs']))
        
    print(outputs_directory)
    with open(outputs_directory+'/EMPLOYED_PARAMETERS_and_RESULTS.pkl', 'wb') as f:
        pickle.dump(results, f)
    done.append(experiment_name)

Processing 2D_double_slit
Processed already!
Processing 2D_harmonic_dance
Processed already!
Processing 2D_harmonic_gs
Processed already!
Processing 2D_harmonic_rot_gs
Processed already!
Processing 2D_torus_collision
Processed already!
Processing 2D_torus_dance
Processed already!
Processing 1D_double_barrier_p3
Processed already!
Processing 1D_double_barrier_p1
Processed already!
Processing 1D_double_barrier_p2
Processed already!
Processing 1D_free_particle_p1
Processed already!
Processing 1D_free_particle_p0
Processed already!
Processing 1D_harmonic_gs
Processed already!
Processing 1D_self_interference
Processed already!
Processing 1D_harmonic_dance
Still not finished!
Processing 3D_harmonic_dance
Still not finished!
Processing 3D_harmonic_gs
Still not finished!
Processing 3D_free_particle_p0
Still not finished!
Processing 3D_hydrogen_2pz
Still not finished!
Processing 3D_hydrogen_gs
Still not finished!
Processing 1D_single_barrier_p1
Processed already!
Processing 1D_single_barrier_p2

# Plot Metrics

In [12]:
def plot_K_A_metric_grid( results, Ks, As, title, save_full_path, interpol="nearest", scale=None):
    # results should be a list ordered as  for K in Ks_to_try:  for A in As_to_try:
    results = cnp.array(results).reshape((len(Ks), len(As))) #[NumKs, NumAs]
    fig=plt.figure(figsize=(7,7))
    ax =fig.add_subplot(111)
    cm = ax.imshow(results, cmap='RdYlBu_r', interpolation=interpol)
    ax.set_xlabel("A")
    ax.set_ylabel("K")
    ax.set_xticks(np.arange(0,len(As)), [f"{A:.3}" for A in As])
    ax.set_yticks(np.arange(0,len(Ks)), [f"{K:.3}" for K in Ks])
    cb=fig.colorbar(cm,  orientation='vertical')
    cb.ax.set_xlabel("Time averaged metric")
    ax.set_title(title)
    if scale is not None:
        ax.set_zscale(scale)
    #plt.show()
    plt.savefig(save_full_path, dpi=120)
    plt.close(fig)
    
def plot_metric_time_evolution( ts, metricss, title, labels, save_full_path, ylabel="Average Error" ):
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111)
    for metrics,ID in zip(metricss, labels):
        ax.plot(ts, metrics, 'o-', label=ID)
    ax.set_xlabel("Time (a.u.)")
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    plt.legend()
    plt.savefig(save_full_path)
    plt.close(fig)

In [7]:
plotted=[]

In [8]:
for experiment_name, grid_file, path_to_settings in zip(experiment_names, experiment_grid_files, settings_files):
    if experiment_name in plotted:
        continue
    outputs_directory = f"./OUTPUTS/{experiment_name}/"
    if not os.path.isfile(f"{outputs_directory}/FINISHED.txt"):
        continue
    with open(outputs_directory+'/EMPLOYED_PARAMETERS_and_RESULTS.pkl', 'rb') as f:
        results = pickle.load(f)
    if len(results['Krange'])>2:
        Ks = results['Krange']
        As = results['Arange']
    else:
        Ks = np.linspace(results['Krange'][0], results['Krange'][1], results['numK'])
        As = np.linspace(results['Arange'][0], results['Arange'][1], results['numA'])
 
    metrics = ['pdf_dif_w_ref', 'traj_dif_w_ref','mom_dif_w_ref', 'mom_expect_dif_c_w_pdf', 'mom_expect_dif_c_w_trajs', 'posit_expect_dif_w_pdf', 'posit_expect_dif_w_trajs']
    plurals = ['pdf_difs_w_ref', 'traj_difs_w_ref','mom_difs_w_ref', 'mom_expect_difs_c_w_pdf', 'mom_expect_difs_c_w_trajs', 'posit_expect_difs_w_pdf', 'posit_expect_difs_w_trajs']
    titles = ['Average Probability Density Abs. Error', 'Average Lagr. Trajectory Abs. Error', 'Average Trajectory Momentum Abs. Error', 'Average Expected Momentum Abs. Error\nComputed in reference using pdf', 'Average Expected Momentum Abs. Error\nComputed in reference using trajs', 'Average Expected Position Abs. Error\nComputed in reference using pdf', 'Average Expected Position Abs. Error\nComputed in reference using trajs']
    file_names = [f'/pdf_error_{experiment_name}.png', f'/traj_error_{experiment_name}.png', f'/mom_error_{experiment_name}.png', f'/expec_mom_error_pdf_{experiment_name}.png', f'/expec_mom_error_trajs_{experiment_name}.png',  f'/expec_posit_error_pdf_{experiment_name}.png', f'/expec_posit_error_trajs_{experiment_name}.png']
    logs = [1,1,1,1,1,1,1]
    for metric, title, file_name, log in zip(metrics, titles, file_names, logs):
        plot_K_A_metric_grid( results['average_metrics_wrt_ref'][metric] if not log else cnp.log(results['average_metrics_wrt_ref'][metric] ),
                    Ks, As, f"{experiment_name}\n{'Log ' if log else ''}{title}", outputs_directory+file_name)
    # now plot the time evolutions of the best 5 cases in each
    Nbest = 5
    file_names = [f'/pdf_error_t_{experiment_name}.png', f'/traj_error_t_{experiment_name}.png', f'/mom_error_t_{experiment_name}.png', f'/expec_mom_error_pdf_t_{experiment_name}.png', f'/expec_mom_error_trajs_t_{experiment_name}.png', f'/expec_posit_error_pdf_t_{experiment_name}.png', f'/expec_posit_error_trajs_t_{experiment_name}.png']
    ts = np.array([j*results['dt_SE'] for j in range(results['numIts_SE'])])
    output_ts = ts[::results['outputEvery_SE']]
    for metric, plural, title, file_name in zip(metrics, plurals, titles, file_names):
        outs = results['average_metrics_wrt_ref'][metric]
        idxs_to_plot = np.argsort(outs)[:Nbest]
        to_plot_IDs=[]
        to_plot_metrics=[]
        for idx in idxs_to_plot:
            to_plot_IDs.append(results['Experiment_IDs'][idx]+f' Avg={outs[idx]:.3}')
            to_plot_metrics.append(results[results['Experiment_IDs'][idx]][plural])
        plot_metric_time_evolution( output_ts, to_plot_metrics, title,to_plot_IDs,
                        save_full_path=outputs_directory+file_name )
        plotted.append(experiment_name)

## Ad-hoc Metrics
### Transmission from a certain point on

In [8]:
def compute_transmission_metric_trajs(trajs, boundary, dim):
    # "boundary" is the point above which the particles will be
    # considered transmitted, along the axis "dim"
    return cnp.sum(trajs[:,dim]>=boundary)/len(trajs)
def compute_transmission_metric_pdf(grid, pdf, boundary, dim):
    return cnp.where(grid[dim]>=boundary, pdf, 0).sum()/pdf.sum()

In [9]:
done_transmission_experiment_queue_path = "./QUEUE_transmission_done.txt"
experiment_names = []
experiment_grid_files = []
settings_files = []
boundaries = []
cutoff_dims = []
with open(done_transmission_experiment_queue_path, 'r') as f:
    tasks = f.readlines()
for task in tasks:
    args = task.replace(" ", "").split('(')[-1].split(')')[0].split(',')
    experiment_names.append(args[0])
    experiment_grid_files.append(args[-4])
    settings_files.append(args[1])
    boundaries.append(float(args[-2]))
    cutoff_dims.append(int(args[-1]))
print(experiment_names)
print(experiment_grid_files)
print(settings_files)
print(boundaries)
print(cutoff_dims)

['2D_double_slit', '2D_single_slit_p1', '2D_single_slit_p2', '2D_single_slit_p3', '1D_single_barrier_p1', '1D_single_barrier_p2', '1D_single_barrier_p3', '1D_double_barrier_p1', '1D_double_barrier_p2', '1D_double_barrier_p3']
['./SIMULATION_GRID_SETTINGS/SETTINGS_Grid_2d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_2d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_2d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_2d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_1d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_1d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_1d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_1d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_1d.txt', './SIMULATION_GRID_SETTINGS/SETTINGS_Grid_1d.txt']
['./SIMULATION_SCENARIO_SETTINGS/2D/SETTINGS_2d_double_slit.txt', './SIMULATION_SCENARIO_SETTINGS/2D/SETTINGS_2d_single_slit_p1.txt', './SIMULATION_SCENARIO_SETTINGS/2D/SETTINGS_2d_single_slit_p2.txt', './SIMULATION_SCENARIO_SETTINGS/2D/SETTINGS_2d_single_slit_p3

In [10]:
for experiment_name, grid_file, path_to_settings, boundary, dim in zip(experiment_names, experiment_grid_files, settings_files, boundaries, cutoff_dims):
    outputs_directory = f"./OUTPUTS/{experiment_name}/"
    if not os.path.isfile(f"{outputs_directory}/FINISHED.txt"):
        continue
    try:
        with open(outputs_directory+'/EMPLOYED_PARAMETERS_and_RESULTS.pkl', 'rb') as f:
            results = pickle.load(f)
        loaded=True
    except:
        loaded=False
        results = {}
        results['Experiment_IDs'] = []
        with open(f"{grid_file}", "r") as f:
            results['numK'] = int(f.readline().split("numK ")[1])
            results['numA'] = int(f.readline().split("numA ")[1])
            results['Krange'] = [float(x) for x in f.readline().split("Krange ")[1].split('[')[1].split(']')[0].split(',')]
            results['Arange'] = [float(x) for x in f.readline().split("Arange ")[1].split('[')[1].split(']')[0].split(',')]
            dim = int(f.readline().split("dim ")[1])
        results['average_metrics_wrt_ref'] = {}
        with open(f"{path_to_settings}", "r") as f:
            results['numIts_SE'] = int(f.readline().split("numIts_SE ")[1])
            results['dt_SE'] = float(f.readline().split("dt_SE ")[1])
            results['outputEvery_SE'] = int(f.readline().split("outputEvery_SE ")[1])
            results['Ns_pdf'] = [int(x) for x in f.readline().split("Ns_SE_and_Newt_pdf ")[1].split('[')[1].split(']')[0].split(',')]
            results['xlowers'] = [float(x) for x in f.readline().split("xlowers ")[1].split('[')[1].split(']')[0].split(',')]
            results['xuppers'] = [float(x) for x in f.readline().split("xuppers ")[1].split('[')[1].split(']')[0].split(',')]
            results['numDofUniv'] = int(f.readline().split("numDofUniv ")[1])
            results['numDofPartic'] = int(f.readline().split("numDofPartic ")[1])
            results['numTrajs'] = int(f.readline().split("numTrajs ")[1])
            results['hbar'] = float(f.readline().split("hbar ")[1])
            results['ms'] = [float(x) for x in f.readline().split("ms ")[1].split('[')[1].split(']')[0].split(',')]
            results['complex_dtype'] = f.readline().split("complex_dtype ")[1]
            results['real_dtype'] =  f.readline().split("real_dtype ")[1]
            results['K_coulomb'] = float(f.readline().split("K_coulomb ")[1])
            results['qs'] = [float(x) for x in f.readline().split("qs ")[1].split('[')[1].split(']')[0].split(',')]
            results['dt_MIW'] = float(f.readline().split("dt_Newt ")[1])
            results['Ns_potential'] = [int(x) for x in f.readline().split("Ns_Newt_pot ")[1].split('[')[1].split(']')[0].split(',')]
            results['use_Coulomb_potential'] = bool(int(f.readline().split("use_Coulomb_potential_Newt ")[1]))
            results['use_scenario_potential'] = bool(int(f.readline().split("use_scenario_potential_Newt ")[1]))
            results['sigma'] = float(f.readline().split("sigma_pdf_estimation ")[1])
            
    if len(results['Krange'])>2:
        Ks_to_try = results['Krange']
        As_to_try = results['Arange']
    else:
        Ks_to_try = np.linspace(results['Krange[0]'], results['Krange[1]'], results['numK'])
        As_to_try = np.linspace(results['Arange[0]'], results['Arange[1]'], results['numA'])
    for K in Ks_to_try:
        for A in As_to_try:
            ID = f"K_{K:.4}_A_{A:.4}"
            if not loaded:
                results['Experiment_IDs'].append( ID )
                results[ID] = {}
                results[ID]['K'] = K
                results[ID]['A'] = A
            results[ID]['transmission_in_time'] = []
            results[ID]['transmission_in_time_dif_w_ref_pdf'] = []
            results[ID]['transmission_in_time_dif_w_ref_trajs'] = []
    results['average_metrics_wrt_ref']['transmission_dif_w_ref_pdf']=[]
    results['average_metrics_wrt_ref']['transmission_dif_w_ref_trajs']=[]
    results['transmission_in_time_pdf_ref'] = []
    results['transmission_in_time_trajs_ref'] = []
    
    nodes = [cnp.linspace(results['xlowers'][j], results['xuppers'][j], results['Ns_pdf'][j]) for j in range(results['numDofUniv'])] # (xs, ys)
    if results['numDofUniv']==3:
        grid=cnp.moveaxis(cnp.array(np.meshgrid(*nodes)).swapaxes(1,2),0,-1) #[Nx,Ny,Nz,3]
    elif results['numDofUniv']==2:
        grid=cnp.array(cnp.meshgrid(*nodes)).T #[Nx,Ny, 2]
    elif results['numDofUniv']==1:
        grid=nodes[0][:,cnp.newaxis]
    grid = cnp.moveaxis(grid, -1, 0) #[3, Nx, Ny, Nz]
    cxlowers = cnp.array(results['xlowers'])[cnp.newaxis, :]
    cxuppers = cnp.array(results['xuppers'])[cnp.newaxis, :]
    results['dxs'] = [(results['xuppers'][j]-results['xlowers'][j])/(results['Ns_pdf'][j]-1) for j in range(results['numDofUniv'])] # (dx, dy)
    cdxs = cnp.array(results['dxs'])[cnp.newaxis, :]
    
    # Match the output number
    outputEvery = int(results['dt_SE']//results['dt_MIW'])
    if not loaded:
        # now match exactly the outputed frames
        results['outputEvery_MIW'] = results['outputEvery_SE']*outputEvery
        results['numIts_MIW'] = int(results['numIts_SE']*(results['dt_SE']//results['dt_MIW']))

    its_SE = np.array([j for j in range(results['numIts_SE'])])
    output_its_SE = its_SE[::results['outputEvery_SE']]
    its_MIW = np.array([j for j in range(results['numIts_MIW'])])
    output_its_MIW = its_MIW[::results['outputEvery_MIW']]
    for it_SE, it_MIW in zip(output_its_SE, output_its_MIW):
        # take the reference results
        se_simul_path = outputs_directory+f"/SE_{results['numDofUniv']}D/Reference_SE_Simulation/"
        pdf_ref = np.load(se_simul_path+f"/pdf/pdf_it_{it_SE}.npy") #[Nx,Ny,Nz]
        trajs_ref = np.load(se_simul_path+f"/trajs/trajs_it_{it_SE}.npy") #[numTrajs, 2*dimUniv]
        #mom_field_ref = np.load(se_simul_path+f"/moms/momentum_field_it_{it_SE}.npy") #[Nx,Ny, Nz, 3]
        # remove overflows
        #over = 10
        #mom_field_ref = cnp.nan_to_num( mom_field_ref, nan=0, posinf=over, neginf=-over )
        results['transmission_in_time_trajs_ref'].append(compute_transmission_metric_trajs(trajs_ref[:,:results['numDofUniv']], boundary, dim))
        results['transmission_in_time_pdf_ref'].append(compute_transmission_metric_pdf(grid, pdf_ref, boundary, dim))
        for ID in results['Experiment_IDs']:
            miw_simul_path = outputs_directory + f"/MIW/{ID}/"
            #pdf_miw = np.load(miw_simul_path+f"/pdf/pdf_it_{it_MIW}.npy") #[Nx,Ny,Nz]
            trajs_miw = np.load(miw_simul_path+f"/trajs/trajs_it_{it_MIW}.npy") #[numTrajs, 2*dimUniv]
            results[ID]['transmission_in_time'].append(compute_transmission_metric_trajs(trajs_miw[:,:results['numDofUniv']], boundary, dim))
            results[ID]['transmission_in_time_dif_w_ref_pdf'].append(np.abs(results[ID]['transmission_in_time'][-1]-results['transmission_in_time_pdf_ref'][-1]))
            results[ID]['transmission_in_time_dif_w_ref_trajs'].append(np.abs(results[ID]['transmission_in_time'][-1]-results['transmission_in_time_trajs_ref'][-1]))
    for ID in results['Experiment_IDs']:
        results['average_metrics_wrt_ref']['transmission_dif_w_ref_pdf'].append( np.mean(results[ID]['transmission_in_time_dif_w_ref_pdf']))
        results['average_metrics_wrt_ref']['transmission_dif_w_ref_trajs'].append( np.mean(results[ID]['transmission_in_time_dif_w_ref_trajs']))

    #print(results)
    with open(outputs_directory+'/EMPLOYED_PARAMETERS_and_RESULTS.pkl', 'wb') as f:
        pickle.dump(results, f)

In [14]:
for experiment_name, grid_file, path_to_settings in zip(experiment_names, experiment_grid_files, settings_files):
    outputs_directory = f"./OUTPUTS/{experiment_name}/"
    if not os.path.isfile(f"{outputs_directory}/FINISHED.txt"):
        continue
    with open(outputs_directory+'/EMPLOYED_PARAMETERS_and_RESULTS.pkl', 'rb') as f:
        results = pickle.load(f)
    if len(results['Krange'])>2:
        Ks = results['Krange']
        As = results['Arange']
    else:
        Ks = np.linspace(results['Krange'][0], results['Krange'][1], results['numK'])
        As = np.linspace(results['Arange'][0], results['Arange'][1], results['numA'])
 
    metrics = ['transmission_dif_w_ref_pdf', 'transmission_dif_w_ref_trajs']
    plurals = ['transmission_in_time_dif_w_ref_pdf', 'transmission_in_time_dif_w_ref_trajs']
    titles = ['Transmission Difference\nUsing pdf of reference', 'Transmission Difference\nUsing trajectories of reference']
    file_names = [f'/transmission_dif_pdf_{experiment_name}.png', f'/transmission_dif_trajs_{experiment_name}.png']
    logs = [0]*2
    for metric, title, file_name, log in zip(metrics, titles, file_names, logs):
        plot_K_A_metric_grid( results['average_metrics_wrt_ref'][metric] if not log else cnp.log(results['average_metrics_wrt_ref'][metric] ),
                    Ks, As, f"{experiment_name}\n{'Log ' if log else ''}{title}", outputs_directory+file_name)
    # now plot the time evolutions of the best 5 cases in each
    Nbest = 5
    file_names = [f'/transmission_error_pdf_t_{experiment_name}.png', f'/transmission_error_trajs_t_{experiment_name}.png', f'/mom_error_t_{experiment_name}.png', f'/expec_mom_error_pdf_t_{experiment_name}.png', f'/expec_mom_error_trajs_t_{experiment_name}.png', f'/expec_posit_error_pdf_t_{experiment_name}.png', f'/expec_posit_error_trajs_t_{experiment_name}.png']
    ts = np.array([j*results['dt_SE'] for j in range(results['numIts_SE'])])
    output_ts = ts[::results['outputEvery_SE']]
    for metric, plural, title, file_name in zip(metrics, plurals, titles, file_names):
        outs = results['average_metrics_wrt_ref'][metric]
        idxs_to_plot = np.argsort(outs)[:Nbest]
        to_plot_IDs=[]
        to_plot_metrics=[]
        for idx in idxs_to_plot:
            to_plot_IDs.append(results['Experiment_IDs'][idx]+f' Avg={outs[idx]:.3}')
            to_plot_metrics.append(results[results['Experiment_IDs'][idx]][plural])
        plot_metric_time_evolution( output_ts, to_plot_metrics, title,to_plot_IDs,
                        save_full_path=outputs_directory+file_name )
    to_plot_IDs=["SE Reference pdf", "SE Reference trajs"]
    to_plot_metrics=[results['transmission_in_time_pdf_ref'], results['transmission_in_time_trajs_ref']]
    for idx in idxs_to_plot:
        to_plot_IDs.append(results['Experiment_IDs'][idx]+f' Avg Error={outs[idx]:.3}')
        to_plot_metrics.append(results[results['Experiment_IDs'][idx]]['transmission_in_time'])
    plot_metric_time_evolution( output_ts, to_plot_metrics, 'Transmission in time comparison',to_plot_IDs,
                    save_full_path=outputs_directory+f'/transmission_t_{experiment_name}.png' )